<a href="https://colab.research.google.com/github/vbaezfortunato/Python_class/blob/master/Lab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [103]:
import pandas as pd
import numpy as np
import datetime
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from google.colab import files
import spacy
import spacy.cli
spacy.cli.download("es_core_news_sm")
uploaded = files.upload()
uploaded1 = files.upload()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')


Saving es_spam.csv to es_spam (3).csv


Saving imdb_rev.txt to imdb_rev (2).txt


In [104]:
nlp = spacy.load('es_core_news_sm')

# **Problema 1:**

Utilizando el codigo trabajado durante la sesion de laboratorio construya una funcion que le permita realizar la clasicacion de un texto aleatorio proporcinado como argumento a dicha funcion. Su funcion debe utilzar los procedimientos y modelos de Machine Learning que se generaron en clase, si usted desea puede reescribir los procedimientos para volverlos mas eficientes, el objetivo es que se pueda realizar la prediccion de ham o spam para cualquier string dado.

In [105]:
spamDB = pd.read_csv('es_spam.csv', error_bad_lines=False, sep=",",  names=['label', 'message'])
spamDB = spamDB.iloc[1:]
spamDB

,label,message
1,ham,"Ir hasta el punto de jurong, loco .. Disponibl..."
2,ham,lar bien ... Bromas WIF u oni ...
3,spam,Entrada libre en una imagen de obsequio 2 wkly...
4,ham,T Dun decir hor tan temprano ... t r ya contin...
5,ham,"Nah no creo que vaya a USF, que vive por aquí,..."
...,...,...
5568,spam,Esta es la segunda vez que hemos intentado 2 d...
5569,ham,Se √º b explanada de ir a casa fr?
5570,ham,"Lástima, * estaba en el estado de ánimo para e..."
5571,ham,El chico hizo un poco de putear pero actuaba c...


In [106]:
#corpus = spamDB.mensaje
corpus = spamDB.message

spamDB.label.value_counts()/len(spamDB)

ham     0.865937
spam    0.134063
Name: label, dtype: float64

In [107]:

ham = spamDB[spamDB['label'] == 'ham']
spam = spamDB[spamDB['label'] == 'spam']
ham = ham.sample(2*spam.shape[0])
ham.shape, spam.shape

((1494, 2), (747, 2))

In [108]:
dataset = ham.append(spam, ignore_index=True)
dataset.shape

(2241, 2)

In [109]:

corpus = dataset.message

In [110]:
import re, string
def StrNomralization(corpus):
    newCorpus = []
    for i, doc in enumerate(corpus):
        newCorpus.append(re.sub(r'[^a-zA-Z0-9\s{1}áéíóúü]', '', doc).lower().strip().replace('     ', '').rstrip('\r\n').rstrip("\n"))
    return newCorpus

In [111]:

newTrainCorpus = StrNomralization(corpus)

In [112]:
#tokenizacion
def tokenizationV1(newCorpus):
    documents = []
    for i, doc in enumerate(newCorpus):
        documents.append(nlp(doc))
    return documents

newTrainCorpus = tokenizationV1(newTrainCorpus)

In [113]:
def removeStops(documents):
    newDoc = []
    for index, doc in enumerate(documents):
        s = ""
        for token in doc:
            if ((token.is_stop == False) ): #√
                s = s + token.text + " "
          #print(s)
        newDoc.append(s.strip())
    return newDoc

newTrainCorpus = removeStops(newTrainCorpus)

In [114]:

#stemming y lematizacion

def stemmingLemmating(newDoc):
    documents = []
    for doc in newDoc:
        #print(doc)
        documents.append(nlp(doc))


    newDoc = []
    for index, doc in enumerate(documents):
        s = ""
        for token in doc:
            s = s + token.lemma_ + " "
        newDoc.append(s.strip())
    return newDoc

readyTrainCorpus = stemmingLemmating(newTrainCorpus)
newTrainCorpus = stemmingLemmating(newTrainCorpus)

In [115]:
def listToString(s):  
    str1 = ""  
    for ele in s:  
        str1 += ele + " "
    return str1

In [116]:
def removeTrash(newDoc):
    mainStr = listToString(newDoc).strip().split(' ')
    #mainStr = [x for x in mainStr if (len(x) >= 0)] #camviar esto
    return mainStr

newTrainCorpus = removeTrash(newTrainCorpus)

In [117]:
newTrainCorpus = set(newTrainCorpus)

In [118]:
trainColumns = list(newTrainCorpus)
trainRows = range(0, len(readyTrainCorpus))

In [119]:
def generateEmptyTf(cols, rows):
    outDf = pd.DataFrame(index=rows, columns=cols)
    outDf = outDf.fillna(0) # with 0s rather than NaNs

    return outDf

tfTrain = generateEmptyTf(trainColumns, trainRows)

In [120]:
def calculoTf(newDoc, df):
    documents = []

    for doc in newDoc:
        documents.append(nlp(doc))
    for index, doc in enumerate(documents):
        bagOfWordsLen = len(doc)
        for word in doc:
            try:
                colIndex = list(df.columns).index(word.text)
                #print(colIndex)
                df.iloc[index, colIndex] = df.iloc[index, colIndex] + 1
                #termFreqTemp.iloc[index, colIndex] = termFreqTemp.iloc[index, colIndex] + 1
            except Exception as e: 
            #print(e)
            #print("error")
            #print(doc)
                pass
        df.iloc[index, :] = df.iloc[index, :]/bagOfWordsLen

    #for index, doc in enumerate(documents):
    #  df.iloc[index, :] = df.iloc[index, :]/df.shape[1]
    return df


trainTf = calculoTf(readyTrainCorpus, tfTrain)

In [121]:
def calculoIdf(df):
    N = df.shape[0]
    #print(N)
    valX = (N/df.astype(bool).sum(axis=0))
    #print(df.astype(bool))
    idfValue = pd.Series(np.log(valX))
    return idfValue

trainIdf = calculoIdf(trainTf)

In [122]:
#calculo tf-idf
tfIdfTrain = trainTf.mul(trainIdf, axis=1)
tfIdfTrain = tfIdfTrain.fillna(0)
tfIdfTrain

,,oir,ppt150x3,09066612661,1327,bb,grupo,sentir,lacsthere,oscuridad,alondra,euro2004,beforewent,recordar,renon,proporcionar,mirar,agidhane,footbl,necesitar,3510i,anuncio,melodía,ah,vivekanand,especial,siglo,estabilidad,xxxxxxx,payed2day,sarasota,fundamental,cashbincouk,cama,velar,txt,concertar,yeh,864233,walmart,...,streamate,900,compensar,mising,gimnasio,frotar,netcollex,diario,danos,o2,abundante,lucozade,explícito,fauna,filosófico,asia,golpear,shud,mmm,2moro,workin,seriamente,or2optout,ill,nombre,rang,tkts,thm,150p16,21,hijo,nojst,toar,breaker,estribo,vry,89105,ar,estacionamiento,tming
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.82699,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2236,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.352202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2238,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2239,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25318,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [123]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tfIdfTrain, dataset['label'], test_size=0.3, random_state=0, shuffle=True)

In [124]:
def encode(x):
    if(x == "ham"):
        return 0
    else:
        return 1

In [125]:
y_train = list(map(encode, y_train))

In [126]:
from sklearn.svm import SVC

svmc = SVC(kernel='linear')
svmc.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [127]:

X_text = X_test.fillna(0)
y_pred_svm = svmc.predict(X_test)

In [128]:

from sklearn.metrics import confusion_matrix, accuracy_score
y_test = list(map(encode, y_test))
confusion_matrix(y_test, y_pred_svm)

array([[436,  25],
       [ 15, 197]])

In [129]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred_svm)

0.9405646359583952

In [130]:

def decode(x):
  if(x == 0):
    return "ham"
  else:
    return "spam"

In [131]:
def PrediceSpam(texto):
    Normaliza = StrNomralization(texto)
    NuevoCorpus = tokenizationV1(Normaliza)
    NuevoCorpus = removeStops(NuevoCorpus)
    readyTrainCorpus = stemmingLemmating(NuevoCorpus)
    NuevoCorpus = stemmingLemmating(NuevoCorpus)
    NuevoCorpus = removeTrash(NuevoCorpus)
    NuevoCorpus = set(NuevoCorpus)
    
    Columns = list(NuevoCorpus)
    
    #Se obtienen los pesos de la frase por palabra
    tfTrain = generateEmptyTf(Columns, [1])
    trainTf = calculoTf(readyTrainCorpus, tfTrain)
    
    #Se obtiene una row en blanco de la longitud de los dataset de training
    tfTrainGeneral = generateEmptyTf(trainColumns, [1])
    
    for i in Columns:
        tfTrainGeneral[i]=trainTf[i]
    
    y_pred_texto = svmc.predict(tfTrainGeneral)
    
    return decode(y_pred_texto[0]), trainIdf

In [132]:
Prediccion, matriz = PrediceSpam(['Por favor llame a nuestro representante de servicio al cliente en el 0800 169 6031 entre las 10 a.m.-9 p.m. a medida que han ganado un premio garantizado de 1.000 en efectivo o  5.000 premio!'])

In [133]:
Prediccion

'spam'

## **Problema 2:**

Para el dataset llamado imdb rev.txt desarrolle un clasicador de reviews positivos o negativos, la variable que debe predecir esta codificada como 0 y como 1, donde 0 es un review malo y 1 es un review bueno. Tome en cuenta que este archivo no es un csv y usted debera procesarlo para poder formar una estructura adecuada para la clasificacion, ademas note que esta en ingles, por tal motivo debe utilizar el modelo de lenguaje adecuado. Recuerde usar todos los recursos que conoce sobre Machine Learning para determinar si su programa clasifica adecuadamente, puede usar cualquier clasificador que desee incluso varios si lo considera necesario

In [134]:
lineas = []

archivo = open("imdb_rev.txt", "r")
for linea in archivo.readlines():
    lineas.append(linea)
archivo.close()

In [135]:
datos = pd.DataFrame(lineas,columns=['Review'])

In [136]:
datos['Calification']=(datos['Review'].str.slice(start=-2)).str.strip()
datos['Review'] = datos['Review'].str.slice(start = 0, stop = -3)

In [137]:
datos


,Review,Calification
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1
...,...,...
995,I just got bored watching Jessice Lange take h...,0
996,"Unfortunately, any virtue in this film's produ...",0
997,"In a word, it is embarrassing.",0
998,Exceptionally bad!,0


In [138]:
corpus = []
for i in range(0, 1000):
    review = re.sub('[^a-zA-Z]', ' ', datos['Review'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')
    review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
    review = ' '.join(review)
    corpus.append(review)

In [139]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(corpus).toarray()
y = datos.iloc[:, -1].values

In [140]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [141]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [142]:

y_pred = classifier.predict(X_test)

In [143]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[71 15]
 [50 64]]


0.675

In [151]:
corpus = []
for i in range(0, 1000):
    review = re.sub('[^a-zA-Z]', ' ', datos['Review'][i])
    review = review.lower()
    review = review.split()
    sb = SnowballStemmer(language = 'english')
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')
    review = [sb.stem(word) for word in review if not word in set(all_stopwords)]
    review = ' '.join(review)
    corpus.append(review)

In [153]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(corpus).toarray()
y = datos.iloc[:, -1].values

In [154]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [155]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [156]:
y_pred = classifier.predict(X_test)

In [157]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[71 15]
 [49 65]]


0.68